In [1]:
#Created: 15Apr2022
#Author: Lynn Menchaca

#Resources:
#Kaggle Competition-> Spaceship Titanic

#Purpose: Clean and Analyze Data File

In [30]:
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
data_file_path = '/Users/lynnpowell/Documents/DS_Projects/Spaceship_Titanic/'
df = pd.read_csv(data_file_path+'train.csv')

df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [6]:
df.shape

(8693, 14)

In [7]:
#find all the missing values in each column
df.isnull().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

In [17]:
#Looking at how many rows have multiple missing data columns
df.isnull().sum(axis=1).value_counts()
#df[df.isnull().sum(axis=1)==3]

0    6606
1    1867
2     203
3      17
dtype: int64

In [28]:
#Removing name column since it is similar to the PassengerID column
#Removing name column also because it has missing data
df = df.drop('Name',axis=1)

In [29]:
#Just for now I am just going to remove all the missing data to see what columns
#have the highest correlation values
df_full = df.dropna()
df_full.shape

(6764, 13)

In [36]:
#Decription of Ages
df_full['Age'].describe()

count    6764.000000
mean       28.897398
std        14.529563
min         0.000000
25%        19.000000
50%        27.000000
75%        38.000000
max        79.000000
Name: Age, dtype: float64

In [27]:
#Columns with categorical data that need dummy values
print(df_full[['HomePlanet']].value_counts())
print('\n')
print(df_full[['Destination']].value_counts())

HomePlanet
Earth         3566
Europa        1673
Mars          1367
dtype: int64


Destination  
TRAPPIST-1e      4576
55 Cancri e      1407
PSO J318.5-22     623
dtype: int64
